# 🦌 ELK Simulation 🐺

`IPyElk` can serve as a rich, interactive frontend for complex systems that change over
time.

In [ ]:
import json
import math
import re
from datetime import datetime, timedelta
from pathlib import Path

import ipywidgets as W
import networkx
from IPython.display import display
from numpy.random import normal, uniform

import ipyelk
import ipyelk.nx
import ipyelk.tools
from ipyelk import Elk
from ipyelk.diagram import layout_options
from ipyelk.diagram.elk_model import ElkLabel, ElkNode, ElkPort

In [ ]:
earth, grass, deer, wolves, corpses, poop = populations = [
    W.IntSlider(value=v, description=d, min_value=0, max_value=1000)
    for d, v in {"🌎": 10, "🌱": 60, "🦌": 30, "🐺": 2, "💀": 5, "💩": 5}.items()
]
history = []
play = W.Play(description="Simulate", interval=1000)

Date related-things.

In [ ]:
date = W.DatePicker(value=datetime.now(), description="📆")
months = [(datetime(month=m, year=2020, day=1).strftime("%b"), m) for m in range(1, 13)]

For some visualization, we'll use [vega](https://vega.github.io).

In [ ]:
history = []


def update_history():
    d = date.value.strftime("%b %d %Y")
    history.extend(
        [{"Date": d, "Name": p.description, "Population": p.value} for p in populations]
    )

In [ ]:
def get_vega_spec():
    vega = {
        "$schema": "https://vega.github.io/schema/vega-lite/v3.json",
        "description": "Populations over time",
        "width": "650",
        "data": {"values": history},
        "mark": "line",
        "encoding": {
            "x": {"field": "Date", "type": "temporal"},
            "y": {"field": "Population", "type": "quantitative"},
            "color": {"field": "Name", "type": "nominal"},
        },
    }
    return {"application/vnd.vegalite.v3+json": vega}

In [ ]:
out = W.Output(layout=dict(height="260px", width="800px"))

In [ ]:
def update_plot():
    out.clear_output()
    with out:
        display(get_vega_spec(), raw=True)

In [ ]:
if __name__ == "__main__":
    update_plot()
    display(out)

In [ ]:
def square_label(txt):
    d = math.ceil(math.sqrt(len(txt)))
    return re.sub(f"(.{{,{d}}})", r"\1\n", txt).strip().splitlines()

Build up the new elk graph from the poulations.

In [ ]:
def make_graph():
    graph = networkx.MultiDiGraph()
    [
        graph.add_node(
            p.description,
            labels=[
                ElkLabel(
                    id=f"l_{p.description}_{i}",
                    text=line,
                    layoutOptions={
                        layout_options.NodeLabelPlacement.identifier: "H_CENTER V_CENTER INSIDE",
                    },
                )
                for i, line in enumerate(square_label(p.description * p.value))
            ],
            layoutOptions={
                layout_options.NodeSizeConstraints: "NODE_LABELS FORCE_TABULAR_NODE_LABELS",
                layout_options.NodeLabelPlacement.identifier: "H_CENTER V_CENTER",
            },
        )
        for p in populations
        if p.value
    ]

    [
        graph.add_edge(
            eaten.description,
            eater.description,
            id=f"e_{eater.description}_eats_{eaten.description}",
            labels=[
                ElkLabel(
                    text=text,
                    id=f"l_{eater.description}_eats_{eaten.description}",
                    layoutOptions={
                        layout_options.edge_options.InlineEdgeLabels.identifier: "true"
                    },
                )
            ],
        )
        for eaten, text, eater in [
            [grass, "eaten by", deer],
            [deer, "eaten by", wolves],
            [deer, "becomes", corpses],
            [wolves, "make", poop],
            [deer, "make", poop],
            [corpses, "eaten by", wolves],
            [wolves, "becomes", corpses],
            [corpses, "decomposes into", earth],
            [poop, "decomposes into", earth],
            [earth, "grows", grass],
        ]
        if eater.value and eaten.value
    ]
    return graph

In [ ]:
graph = make_graph()
elk = ipyelk.ElkDiagram()
xelk = ipyelk.nx.XELK(source=(graph, None), label_key="labels")
xelk.layouts[None]["parents"].update(
    {
        layout_options.EdgeRouting.identifier: "SPLINES",
        layout_options.NodeSizeConstraints.identifier: "NODE_LABELS",
    }
)
xelk.connect(elk)
elk_app = ipyelk.Elk(
    transformer=xelk,
    layout=dict(display="flex", flex="1"),
    style={
        " rect.elknode": {
            "stroke": "transparent !important",
            "fill": "transparent !important",
        },
        " .sprotty-edge": {"font-weight": "bold"},
    },
)

Add simulation behaviors.

In [ ]:
knobs = []

In [ ]:
sprout_season = W.SelectionRangeSlider(description="🌱📆", value=(3, 10), options=months)
sprout_rate = W.IntSlider(10, description="🌱📶")
knobs += [sprout_season, sprout_rate]

In [ ]:
def grass_tick():
    start, end = sprout_season.value
    if start <= date.value.month <= end:
        grass.value += sprout_rate.value
        earth.value -= 1

In [ ]:
deer_appetite = W.FloatSlider(0.15, description="🦌🍽️")
fawn_season = W.SelectionRangeSlider(description="🦌📆", value=(4, 7), options=months)
fawn_rate = W.FloatSlider(0.25, description="🦌🍼")
knobs += [deer_appetite, fawn_season, fawn_rate]

In [ ]:
def deer_tick():
    if not deer.value:
        return
    start, end = fawn_season.value
    if start <= date.value.month <= end:
        new_deer = (deer.value * fawn_rate.value) or 1
        deer.value += new_deer
    eaten = deer.value * deer_appetite.value
    grass.value -= eaten
    poop.value += eaten
    if not grass.value:
        died = deer.value / 4
        deer.value -= died
        corpses.value += died
    if uniform() > 0.8:
        deer.value -= 1
        corpses.value += 1

In [ ]:
wolf_appetite = W.FloatSlider(0.1, description="🐺🍽️")
pup_season = W.SelectionRangeSlider(description="🐺📆", value=(4, 4), options=months)
pup_rate = W.FloatSlider(0.25, description="🐺🍼")
pack_size = W.IntSlider(6, description="🐺🐺", min_value=1)
knobs += [wolf_appetite, pup_season, pup_rate, pack_size]

In [ ]:
def wolf_tick():
    if not wolves.value:
        return
    start, end = pup_season.value
    if start <= date.value.month <= end:
        wolves.value += pup_rate.value
    hungry = wolves.value

    if uniform() > 0.1:
        kills = min(deer.value, wolves.value / pack_size.value)
        deer.value -= kills
        corpses.value += kills
        hungry -= kills * pack_size.value
        poop.value += kills

    if hungry > 0 and corpses.value:
        corpses.value -= 1
        hungry = hungry - pack_size.value
        poop.value += 1

    if hungry > 0:
        corpses.value += 1
        wolves.value -= 1

In [ ]:
def corpses_tick():
    if not corpses.value:
        return
    decayed = corpses.value / 4
    earth.value += decayed
    corpses.value -= decayed

In [ ]:
def poop_tick():
    if not poop.value:
        return
    decayed = poop.value / 2
    earth.value += decayed
    poop.value -= decayed

In [ ]:
def update_graph(change=None):
    xelk.source = (make_graph(), None)

In [ ]:
def tick(*args):
    date.value = date.value + timedelta(days=7)
    poop_tick()
    corpses_tick()
    grass_tick()
    deer_tick()
    wolf_tick()
    update_history()
    update_graph()
    update_plot()

Wire up observers.

In [ ]:
[p.observe(update_graph, "value") for p in populations]
play.observe(tick, "value")

Actually draw the app.

In [ ]:
app = W.HBox(
    [
        W.VBox([play, date, *populations, *knobs]),
        W.VBox([elk_app, out], layout=dict(flex="1")),
    ],
    layout=dict(flex="1", height="100%", min_height="80vh"),
)

In [ ]:
if __name__ == "__main__":
    display(app)
    tick()

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).